In [1]:
import pickle
import pandas as pd
import numpy as np
from copy import deepcopy
from train_model import train_model

In [2]:
train = pd.read_pickle('extracted_data/TEST9_global_v4_og_and_norm_weights_train')
val = pd.read_pickle('extracted_data/TEST9_global_v4_og_and_norm_weights_val')
meas = pd.read_pickle('extracted_data/TEST9_global_v4_og_and_norm_weights_meas')
test = pd.read_pickle('extracted_data/TEST9_global_v4_og_and_norm_weights_test')

In [3]:
filename = 'saved_results/features_and_label_sel2'
model_prefix = 'TEST9_sel2'
with open(filename, 'rb') as file:
    features = pickle.load(file)

In [ ]:
depths = [1,2,4,5,6,7,9,11,13,15]

for depth in depths:
    train_model(depth, train, val, features, 'saved_models/'+model_prefix+'_depth_{}'.format(depth), 'saved_history/'+model_prefix+'_depth_{}'.format(depth))
    